# WetGrass analyzed using Edward. Inferring prob of some nodes conditioned on other nodes having given states.

References
* https://discourse.edwardlib.org/t/simple-bayesian-network-sprinkler-example/258

In [1]:
import numpy as np
import tensorflow as tf

import edward as ed
import edward.models as edm
import edward.inferences as edi

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import os
import sys
cur_dir_path = os.getcwd()
print(cur_dir_path)
os.chdir('../../')
qfog_path = os.getcwd()
print(qfog_path)
sys.path.insert(0,qfog_path)

/home/bob/Notebooks/Quantum/quantum-fog/jupyter-notebooks/inference_via_ext_software
/home/bob/Notebooks/Quantum/quantum-fog


In [3]:
# build BayesNet object bnet from bif file
import importlib
mm = importlib.import_module("jupyter-notebooks.inference_via_ext_software.ModelMaker")
from graphs.BayesNet import *
in_path = "examples_cbnets/WetGrass.bif"
bnet = BayesNet.read_bif(in_path, False)

In [4]:
# build model (with no observed nodes) from bnet
file_prefix = "examples_cbnets/WetGrass_obs_none"
mm.ModelMaker.write_edward_model_for_inf(file_prefix, bnet)

.py file with model can be found here

<a href="../../examples_cbnets/WetGrass_obs_none_edward.py">
../../examples_cbnets/WetGrass_obs_none_edward.py</a>

In [5]:
# -i option allows it to access notebook's namespace
%run -i examples_cbnets/WetGrass_obs_none_edward.py

In [6]:
# data
Cloudy_data = tf.constant(0, dtype=tf.int32)
WetGrass_data = tf.constant(1, dtype=tf.int32)

In [7]:
# run
q_dict = {Rain: Rain_q, Sprinkler: Sprinkler_q}
data_dict = {Cloudy: Cloudy_data, WetGrass: WetGrass_data}
inf = edi.KLqp(q_dict, data=data_dict)
inf.run(n_samples = 5, n_iter=1000)

1000/1000 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 1.096


In [8]:
# print final prob distributions

print('\nRain')
print(Rain_q.probs.eval())

print('\nSprinkler')
print(Sprinkler_q.probs.eval())


Rain
[ 0.27310392  0.72689611]

Sprinkler
[ 0.02310836  0.9768917 ]


## Exact results using brute force (enumeration) and junction tree algorithms

In [9]:
from inference.JoinTreeEngine import *
from inference.EnumerationEngine import *
jtree_eng = JoinTreeEngine(bnet)
brute_eng = EnumerationEngine(bnet)

# introduce some evidence
bnet.get_node_named("Cloudy").active_states = [0]
bnet.get_node_named("WetGrass").active_states = [1]

#print node distributiona
node_list = jtree_eng.bnet_ord_nodes
jtree_pot_list = jtree_eng.get_unipot_list(node_list)
brute_pot_list = brute_eng.get_unipot_list(node_list)
for k in range(len(node_list)):
    print("brute:", brute_pot_list[k])
    print("jtree:", jtree_pot_list[k])
    print('')

brute: ['Cloudy']
[ 1.  0.]
jtree: ['Cloudy']
[ 1.  0.]

brute: ['Rain']
[ 0.34839842  0.65160158]
jtree: ['Rain']
[ 0.34839842  0.65160158]

brute: ['Sprinkler']
[ 0.13119789  0.86880211]
jtree: ['Sprinkler']
[ 0.13119789  0.86880211]

brute: ['WetGrass']
[ 0.  1.]
jtree: ['WetGrass']
[ 0.  1.]

